In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('https://drive.google.com/uc?id=1OYMa1QBf6Lt2vVABrFJi8FO4UFhieVRv')
customer = pd.read_csv('https://drive.google.com/uc?id=1uyHiU-iKwJi8ht3TqBqmmFojvSc0CY_N')
transactions = pd.read_csv('transactions_hm.csv')

In [ ]:
transactions.shape

